# 97. U-Net：セグメンテーションの革命的アーキテクチャ

## 学習目標

このノートブックでは、以下を学びます：

1. **U-Net**の構造と設計思想
2. **スキップ接続**の重要性
3. **PyTorchでの実装**
4. **U-Netの応用分野**

## 目次

1. [U-Netとは](#section1)
2. [アーキテクチャの詳細](#section2)
3. [スキップ接続](#section3)
4. [実装](#section4)
5. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch
import japanize_matplotlib

import torch
import torch.nn as nn

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

<a id="section1"></a>
## 1. U-Netとは

### 背景

U-Net（Ronneberger et al., 2015）は、医療画像セグメンテーションのために開発されましたが、今では幅広い分野で使われています。

### 特徴

1. **U字型の構造**：エンコーダとデコーダが対称
2. **スキップ接続**：エンコーダの特徴をデコーダに直接伝達
3. **少ないデータでも高性能**：データ拡張と組み合わせて効果的

In [ ]:
def visualize_unet_architecture():
    """U-Netアーキテクチャの可視化"""
    fig, ax = plt.subplots(figsize=(18, 10))
    
    # カラー設定
    encoder_color = 'lightblue'
    decoder_color = 'lightgreen'
    skip_color = 'lightyellow'
    
    # エンコーダ層（左下へ）
    encoder_configs = [
        (0.05, 0.8, 0.08, 0.15, '64ch'),
        (0.18, 0.65, 0.08, 0.12, '128ch'),
        (0.31, 0.5, 0.08, 0.09, '256ch'),
        (0.44, 0.35, 0.08, 0.06, '512ch'),
    ]
    
    # ボトルネック
    bottleneck = (0.46, 0.2, 0.08, 0.04, '1024ch')
    
    # デコーダ層（右上へ）
    decoder_configs = [
        (0.58, 0.35, 0.08, 0.06, '512ch'),
        (0.71, 0.5, 0.08, 0.09, '256ch'),
        (0.84, 0.65, 0.08, 0.12, '128ch'),
        (0.97, 0.8, 0.08, 0.15, '64ch'),
    ]
    
    # エンコーダ描画
    for x, y, w, h, label in encoder_configs:
        rect = Rectangle((x, y - h/2), w, h, facecolor=encoder_color, edgecolor='blue', linewidth=2)
        ax.add_patch(rect)
        ax.text(x + w/2, y, label, ha='center', va='center', fontsize=9)
    
    # ボトルネック
    x, y, w, h, label = bottleneck
    rect = Rectangle((x, y - h/2), w, h, facecolor='coral', edgecolor='red', linewidth=2)
    ax.add_patch(rect)
    ax.text(x + w/2, y, label, ha='center', va='center', fontsize=9)
    
    # デコーダ描画
    for x, y, w, h, label in decoder_configs:
        rect = Rectangle((x, y - h/2), w, h, facecolor=decoder_color, edgecolor='green', linewidth=2)
        ax.add_patch(rect)
        ax.text(x + w/2, y, label, ha='center', va='center', fontsize=9)
    
    # ダウンサンプリング矢印
    for i in range(3):
        ax.annotate('', xy=(encoder_configs[i+1][0], encoder_configs[i+1][1]),
                   xytext=(encoder_configs[i][0] + encoder_configs[i][2], encoder_configs[i][1]),
                   arrowprops=dict(arrowstyle='->', color='blue', lw=1.5))
    
    # ボトルネックへの矢印
    ax.annotate('', xy=(bottleneck[0], bottleneck[1]),
               xytext=(encoder_configs[-1][0] + encoder_configs[-1][2], encoder_configs[-1][1]),
               arrowprops=dict(arrowstyle='->', color='red', lw=1.5))
    
    # アップサンプリング矢印
    ax.annotate('', xy=(decoder_configs[0][0], decoder_configs[0][1]),
               xytext=(bottleneck[0] + bottleneck[2], bottleneck[1]),
               arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    
    for i in range(3):
        ax.annotate('', xy=(decoder_configs[i+1][0], decoder_configs[i+1][1]),
                   xytext=(decoder_configs[i][0] + decoder_configs[i][2], decoder_configs[i][1]),
                   arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    
    # スキップ接続（水平の矢印）
    for enc, dec in zip(encoder_configs, reversed(decoder_configs)):
        ax.annotate('', xy=(dec[0], dec[1]),
                   xytext=(enc[0] + enc[2], enc[1]),
                   arrowprops=dict(arrowstyle='->', color='orange', lw=2, 
                                  connectionstyle='arc3,rad=0'))
    
    # ラベル
    ax.text(0.25, 0.95, 'エンコーダ\n(Contracting Path)', ha='center', fontsize=12, fontweight='bold')
    ax.text(0.75, 0.95, 'デコーダ\n(Expanding Path)', ha='center', fontsize=12, fontweight='bold')
    ax.text(0.5, 0.08, 'ボトルネック', ha='center', fontsize=11, fontweight='bold')
    
    # 凡例
    from matplotlib.patches import Patch, FancyArrow
    legend_elements = [
        Patch(facecolor=encoder_color, edgecolor='blue', label='エンコーダ'),
        Patch(facecolor=decoder_color, edgecolor='green', label='デコーダ'),
        Patch(facecolor='coral', edgecolor='red', label='ボトルネック'),
    ]
    ax.legend(handles=legend_elements, loc='upper left', fontsize=10)
    
    ax.text(0.5, -0.02, 'オレンジの矢印 = スキップ接続（Skip Connection）', 
           ha='center', fontsize=11, color='orange', fontweight='bold')
    
    ax.set_xlim(0, 1.1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('U-Net アーキテクチャ', fontsize=18, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_unet_architecture()

<a id="section2"></a>
## 2. アーキテクチャの詳細

### エンコーダ（収縮パス）

- 2つの3×3畳み込み + ReLU
- 2×2 Max Poolingでダウンサンプリング
- チャンネル数を2倍に増加

### デコーダ（拡張パス）

- 2×2転置畳み込みでアップサンプリング
- スキップ接続からの特徴と連結（concatenate）
- 2つの3×3畳み込み + ReLU

In [ ]:
def explain_unet_blocks():
    """U-Netの各ブロックを説明"""
    print("="*60)
    print("U-Net の各ブロック")
    print("="*60)
    
    print("""
【エンコーダブロック】
  Input → Conv3×3 → BN → ReLU → Conv3×3 → BN → ReLU → MaxPool2×2 → Output
  
  ・解像度: H×W → H/2 × W/2
  ・チャンネル: C → 2C

【デコーダブロック】
  Input → ConvTranspose2×2 → Concat(skip) → Conv3×3 → BN → ReLU → Conv3×3 → BN → ReLU → Output
  
  ・解像度: H×W → 2H × 2W
  ・チャンネル: C → C/2（スキップ接続後は同じ）

【スキップ接続】
  ・エンコーダの出力をデコーダの入力に直接連結
  ・高解像度の局所的特徴を保持
  ・勾配の流れを改善
    """)

explain_unet_blocks()

<a id="section3"></a>
## 3. スキップ接続

スキップ接続はU-Netの最も重要な要素です。

In [ ]:
def visualize_skip_connection_benefit():
    """スキップ接続の効果を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # サンプル画像（境界が重要）
    np.random.seed(42)
    img = np.zeros((64, 64))
    img[20:45, 15:50] = 1
    
    # 境界をぼやけさせた版
    from scipy.ndimage import gaussian_filter, binary_dilation
    
    # エンコーダの深い特徴（大域的だが粗い）
    deep_feature = gaussian_filter(img.astype(float), sigma=5)
    deep_feature = (deep_feature > 0.3).astype(float)
    
    # 浅い特徴（局所的だが詳細）
    shallow_feature = img.copy()
    
    axes[0].imshow(deep_feature, cmap='Blues')
    axes[0].set_title('深い層の特徴\n（大域的だが境界がぼやける）', fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(shallow_feature, cmap='Oranges')
    axes[1].set_title('浅い層の特徴（スキップ接続）\n（局所的だが境界が鮮明）', fontsize=12)
    axes[1].axis('off')
    
    # 組み合わせ
    combined = (deep_feature + shallow_feature) / 2
    combined = (combined > 0.5).astype(float)
    axes[2].imshow(combined, cmap='Greens')
    axes[2].set_title('組み合わせ\n（大域的コンテキスト + 詳細な境界）', fontsize=12)
    axes[2].axis('off')
    
    plt.suptitle('スキップ接続の効果：詳細な境界情報の保持', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_skip_connection_benefit()

<a id="section4"></a>
## 4. 実装

In [ ]:
class DoubleConv(nn.Module):
    """2つの畳み込みブロック"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    """シンプルなU-Net"""
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(2, 2)
        
        # エンコーダ
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature
        
        # ボトルネック
        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
        
        # デコーダ
        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature * 2, feature, 2, 2))
            self.ups.append(DoubleConv(feature * 2, feature))
        
        # 出力層
        self.final = nn.Conv2d(features[0], out_channels, 1)
    
    def forward(self, x):
        skip_connections = []
        
        # エンコーダ
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        
        # ボトルネック
        x = self.bottleneck(x)
        
        # デコーダ
        skip_connections = skip_connections[::-1]
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)  # アップサンプリング
            skip = skip_connections[idx // 2]
            x = torch.cat([skip, x], dim=1)  # スキップ接続
            x = self.ups[idx + 1](x)  # 畳み込み
        
        return self.final(x)

# モデル作成とテスト
model = UNet(in_channels=3, out_channels=1)
x = torch.randn(1, 3, 256, 256)
y = model(x)

print(f"入力形状: {x.shape}")
print(f"出力形状: {y.shape}")
print(f"パラメータ数: {sum(p.numel() for p in model.parameters()):,}")

<a id="summary"></a>
## 5. まとめ

### U-Netの特徴

1. **U字型構造**: エンコーダで特徴抽出、デコーダで解像度復元
2. **スキップ接続**: 高解像度の詳細情報を保持
3. **対称構造**: 各解像度でスキップ接続

### 応用分野

- 医療画像セグメンテーション
- 衛星画像解析
- 自動運転（道路・車両検出）
- 画像生成（Stable Diffusionの一部）